In [98]:
import pandas as pd
from commcare_export.checkpoint import CheckpointManagerWithDetails
from commcare_export.commcare_hq_client import CommCareHqClient, AUTH_MODE_APIKEY
from commcare_export.commcare_minilinq import get_paginator, PaginationMode
from datetime import datetime
import re

# CommCare credentials
username = 'amutenha@zimttech.org'  # Replace with your actual CommCare username
domain = 'zdip-qa'  # Your CommCare project domain
hq_host = 'https://zdip.zimttech.org'  # Your CommCare HQ custom host
API_KEY = '92fbb3f98433a6929a33015273ebe7b38b6dac39'  # Replace with the API key generated in CommCare


zazicSites= pd.read_excel('zazicFacilities.xlsx')



data = pd.ExcelFile('ZDIPlookup.xlsx')

iqaLookup = pd.read_excel('iqaLookup.xlsx')

# lookupTable = pd.read_excel(data,'previousVMMCSubmissions')

# lookupTable.columns = [col.replace('field: ', '') if col.startswith('field: ') else col for col in lookupTable.columns]

# Initialize API client
api_client = CommCareHqClient(hq_host, domain, username, API_KEY, AUTH_MODE_APIKEY)

# Create a paginator for the 'form' resource (fetching data from a form)
form_paginator = get_paginator(resource='form', pagination_mode=PaginationMode.date_modified, page_size=100)
form_paginator.init()

# Initialize checkpoint manager (None used here for simplicity)
checkpoint_manager = CheckpointManagerWithDetails(None, None, PaginationMode.date_modified)

# Date range for filtering
start_date = datetime(2025,6,25)  # 2 December 2024
end_date = datetime(2025, 12,30)  # 6 December 2024

# Specify the app_id to filter forms
desired_app_id ='0778c330a59625e988e36d5a7d0386b6'  


referrals_data = []
AEs_data = []
general_data = []

# Fetch forms iteratively
# Fetch forms using manual pagination
all_forms = []
offset = 0

params = {
    'limit': 100000,
    'app_id': desired_app_id,
    'received_on_start': start_date.strftime('%Y-%m-%dT%H:%M:%S'),
    'received_on_end': end_date.strftime('%Y-%m-%dT%H:%M:%S')
}

forms = api_client.iterate(
    'form',
    form_paginator,
    checkpoint_manager=checkpoint_manager,
    params=params
)

for form in forms:
    all_forms.append(form)
    if len(all_forms) >= 100000:  # Early stopping if desired
        break

In [99]:
# Print a few fetched forms to verify filtering
for form in all_forms[:5]:  
    pass
    # print(f"Received On: {form.get('received_on')}, App ID: {form.get('app_id')}")

filteredResults =[]
for i in all_forms:
    if i["app_id"] ==desired_app_id:
        filteredResults.append(i)

print(filteredResults)
# Filter and categorize forms by app_id and date
for form in forms:
    if form.get('app_id') == desired_app_id:
        # print(form)
        form_received_date = datetime.strptime(form['received_on'], "%Y-%m-%dT%H:%M:%S.%fZ")
        if start_date <= form_received_date <= end_date:
            general_data.append(form)
# Convert each list to a pandas DataFrame
general_df = pd.json_normalize(general_data)
general_df.to_excel('test.xlsx')

general_df.columns = [re.split(r'[./]', col)[-1] for col in general_df.columns]
desiredColumns = [re.split(r'[./]', col)[-1] for col in iqaLookup.columns]
print("desiredcolumns")
print(desiredColumns)

[{'app_id': '0778c330a59625e988e36d5a7d0386b6', 'archived': False, 'attachments': {'form.xml': {'content_type': 'text/xml', 'length': 3285, 'url': 'https://zdip.zimttech.org/a/zdip-qa/api/form_attachment/v1/7a630bd4-1c3a-4b20-8268-5619a15c09cc/form.xml'}}, 'build_id': 'd5683952cb40438483af32ce61d86e63', 'domain': 'zdip-qa', 'edited_by_user_id': None, 'edited_on': None, 'form': {'#type': 'data', '@name': 'ZAZIC OSDV', '@uiVersion': '1', '@version': '333', '@xmlns': 'http://openrosa.org/formdesigner/72526D77-6087-4854-811D-97DBBA955075', 'activity_and_time_period_under_review': {'section': 'dc'}, 'demand_creation_-_strategies_by_age_bands': {'copy-1-of-yrs': 'OK', 'dc15-24Str1': 'school based', 'dc15-24Str2': '', 'dc15-24Str3': '', 'dc15-24Str4': '', 'dc25-34Str1': 'door to door', 'dc25-34Str2': 'Mobilisation at beerhalls', 'dc25-34Str3': '', 'dc25-34Str4': '', 'demand_creation_-_strategies_by_age_bands': {'dcCommunityCadres': 'Chiefs Headmen', 'dcCommunityCadresComment': 'All Community 

In [100]:
# New columns that should NOT be deleted
columns_to_retain = desiredColumns
# Columns that must be numeric (including the new ones)
numeric_columns = [
    
]

# general_df['AERecordingYear'] = pd.to_numeric(general_df['AERecordingYear'])
# Combine the two lists (retain columns that are in either columns_to_retain or numeric_columns)
# columns_to_keep = list(set(columns_to_retain + numeric_columns))
columns_to_keep = columns_to_retain
# Identify missing columns
missing_columns = [col for col in columns_to_keep if col not in general_df.columns]

In [101]:
# Add missing columns to the DataFrame and assign zero
for col in missing_columns:
    general_df[col] = 0

# Drop the columns that are not in the columns_to_keep list
# Sort the columns alphabetically in both DataFrames

# For the 'general_df', sort the columns alphabetically
general_df_sorted = general_df[columns_to_keep]

# For the AE-specific columns, we can do the same
# ae_columns = [
#     'aeComments', 'ae_type_code', 'date_ae_identified', 'AERecordingYear', 'AERecordingSite', 
#     'client_age', 'circumcising_cadre', 'ae_classification', 'AERecordingMonth', 'mcMethod','vmmc_number'
# ]


# referrals_columns =['ReferrelRecordingMonth', 
#     'ReferrelRecordingSite', 'ReferrelRecordingYear', 'services_referred','reason_for_referral']


# other_referrals = []
# ae_data_df_sorted = general_df[ae_columns][sorted(ae_columns)]
# ae_data_df_sorted = ae_data_df_sorted[ae_data_df_sorted['AERecordingMonth'].notna()]

# referralsDF_sorted = general_df[referrals_columns][sorted(referrals_columns)]
# referralsDF_sorted = referralsDF_sorted[referralsDF_sorted['ReferrelRecordingMonth'].notna()]
general_df_sorted.to_excel("data.xlsx")

In [102]:
# # Function to flatten the nested referral data
# def flatten_referral_data(referral_list):
#     flattened_referrals = []
    
#     # Loop through each referral in the list
#     for referral in referral_list:
#         flattened_referral = {}
        
#         # Flatten the first level of referral data
#         for key, value in referral.items():
#             if isinstance(value, dict):
#                 # If the value is a nested dictionary, flatten it
#                 for nested_key, nested_value in value.items():
#                     flattened_referral[f"{key}_{nested_key}"] = nested_value
#             else:
#                 flattened_referral[key] = value
        
#         flattened_referrals.append(flattened_referral)
    
#     return flattened_referrals



# # Function to unpack nested 'question1_question5' field
# def unpack_detailedField(df, column):
#     if column  in df.columns:
#         # Unpack the dictionary inside 'question1_question5' into separate columns
#         question_fields = df[column].apply(pd.Series)
        
#         # Merge the unpacked fields back into the main DataFrame
#         df = pd.concat([df, question_fields], axis=1)
        
#         # Drop the original 'question1_question5' column if no longer needed
#         df.drop(column, axis=1, inplace=True)
    
#     return df



In [103]:
# # Extract the referrals from 'other_referrals_in_detail' column referrals_data = []
# singleReferrals =[]
# # Loop through each row and check if 'other_referrals_in_detail' is a valid list
# for idx, row in general_df.iterrows():
#     referral_list = row['other_referrals_in_detail']

#     # Check if the value is a list and not NaN
#     if isinstance(referral_list, list):
#         # Flatten the referral data
#         flattened_referrals = flatten_referral_data(referral_list)
#         for ref in flattened_referrals:
#             ref['ReferrelRecordingSite'] =row['facilityName']
#             ref['District'] = row['District']  # Add the District to each referral record
#             referrals_data.append(ref)
#     else:
#         # Handle cases where referral_list is not valid
#         if not pd.isna(row['ReferrelRecordingMonth']):  # Check if 'ReferrelRecordingMonth' is not NaN
#             _object = {
#                 'ReferrelRecordingMonth': row['ReferrelRecordingMonth'], 
#                 'ReferrelRecordingSite': row['ReferrelRecordingSite'], 
#                 'ReferrelRecordingYear': row['ReferrelRecordingYear'],
#                 'District': row['District'],
#                 'reason_for_referral':row['reason_for_referral'],
#                 'services_referred':row['services_referred']
#             }
#             singleReferrals.append(_object)


In [104]:
# aes_data =[]
# singleAEs = []
# for idx, row in general_df.iterrows():
#     ae_list = row['adverse_events']
    
#     # Check if the value is a list and not NaN
#     if isinstance(ae_list, list) and not pd.isna(ae_list).any():  # Use .any() to check the entire list
#         flattened_aes = flatten_referral_data(ae_list)
#         for ref in flattened_aes:
#             ref['District'] = row['District']  # Add the District to each referral record
#             aes_data.append(ref)

#     else:
#         if not pd.isna(row['AERecordingMonth']):
#             _object = {
#                 'AERecordingMonth':row['AERecordingMonth'],
#                 'AERecordingSite':row['AERecordingSite'],
#                 'AERecordingYear':row['AERecordingYear'],
#                 'District':row['District'],
#                 'aeComments':row['aeComments'],
#                 'ae_classification':row['ae_classification'],
#                 'ae_type_code':row['ae_type_code'],
#                 'circumcising_cadre':row['circumcising_cadre'],
#                 'client_age':row['client_age'],
#                 'date_ae_identified':row['date_ae_identified'],
#                 'mcMethod':row['mcMethod'],
#                 'vmmc_number':row['vmmc_number']
#             }
#             singleAEs.append(_object)

In [105]:
# # Convert the flattened referrals data into a DataFrame

# referrals_df = pd.DataFrame(referrals_data)

# # convert the flattened aes data into a df

# ae_df = pd.DataFrame(aes_data)

# # Unpack the 'question1_question5' field
# referrals_df = unpack_detailedField(referrals_df,'question1_question5')

# ae_df = unpack_detailedField(ae_df,'question3_question1')

# ae_df = unpack_detailedField(ae_df,'question1')

# ae_df = ae_df.rename(columns={
#     'question3_AERecordingMonth': 'AERecordingMonth',
#     'question3_AERecordingSite': 'AERecordingSite',
#     'question3_AERecordingYear': 'AERecordingYear'
# })



# if len(singleReferrals)>0:
#     tempRefDF = pd.DataFrame(singleReferrals)
#     referrals_df = pd.concat([referrals_df,tempRefDF])

# if len(singleAEs)>0:
#     tempRefDF = pd.DataFrame(singleAEs)
#     ae_df = pd.concat([ae_df,tempRefDF])


# if not referrals_df.empty:
#     try:
#         referrals_df = referrals_df.drop(columns = ['question1_cancelthisReferralReport',0]).drop_duplicates()
#     except:
#         pass

# if not ae_df.empty:
#     try:
#         ae_df = ae_df.drop(columns = ['checkMonthValidation','checkYearValidation']).drop_duplicates()
#     except:
#         pass

# general_df_sorted = general_df_sorted.drop(columns =['adverse_events','other_referrals_in_detail']).drop_duplicates()


# zazicSites_unique = zazicSites.drop_duplicates(subset='facilityName')


# general_df_sorted = general_df_sorted.loc[:, ~general_df_sorted.columns.duplicated(keep='last')]

# # Merge to bring in facilityType from zazicSites_unique
# general_df_sorted = general_df_sorted.merge(zazicSites_unique[['facilityName', 'facilityType']], 
#                                             on='facilityName', 
#                                             how='left')

# # general_df_sorted['received_on'] = pd.to_datetime(general_df_sorted['received_on']).dt.tz_localize(None)

# # Fill missing values with 0
# general_df_sorted = general_df_sorted.fillna(0)

# # Save the original cleaned DataFrame
# general_df_sorted.to_excel("generalDF.xlsx", index=False)

# # Remove duplicates where formType is 'submission'
# df_filtered = general_df_sorted[~(
#     (general_df_sorted.duplicated(subset=['facilityName', 'recordingMonth', 'year'], keep=False)) &
#     (general_df_sorted['formType'] == 'submission') &
#     (general_df_sorted['facilityName'] != 'Beula')
# )]

# df_filtered = df_filtered[~(
#     (df_filtered['ae_classification'] == 0) & 
#     (df_filtered['facilityName'] == 'Beula')
# )]

# # Keep the latest entry per group based on 'received_on'
# general_df_sorted = df_filtered.loc[
#     df_filtered.groupby(['facilityName', 'recordingMonth', 'year'])['received_on'].idxmax()
# ]

# # general_df_sorted = df_filtered.loc[
# #     df_filtered.groupby(['facilityName', 'recordingMonth', 'year'])['received_on']
# # ]

# # general_df_sorted = df_filtered
# # Sort the final DataFrame for readability
# general_df_sorted = general_df_sorted.sort_values(by=['facilityName', 'year', 'recordingMonth'])

# # Save the final filtered and sorted DataFrame
# general_df_sorted.to_excel("generalDF1.xlsx", index=False)


# # Step 1: Identify the latest 'correction' row for each facilityName, recordingMonth, and year
# latest_correction_idx = general_df_sorted[general_df_sorted['formType'] == 'correction'].groupby(['facilityName', 'recordingMonth', 'year'])['received_on'].idxmax()

# # Step 2: Create a dictionary to store the latest correction rows
# latest_corrections = general_df_sorted.loc[latest_correction_idx]
# # Step 3: Iterate through each submission row and check if there is a corresponding correction row
# for idx, row in general_df_sorted[general_df_sorted['formType'] == 'submission'].iterrows():
#     # Get the corresponding correction row for the current submission
#     correction_row = latest_corrections[
#         (latest_corrections['facilityName'] == row['facilityName']) &
#         (latest_corrections['recordingMonth'] == row['recordingMonth']) &
#         (latest_corrections['year'] == row['year'])
#     ]
    
# # Step 1: Identify the latest 'correction' row for each facilityName, recordingMonth, and year
# latest_correction_idx = general_df_sorted[general_df_sorted['formType'] == 'correction'].groupby(['facilityName', 'recordingMonth', 'year'])['received_on'].idxmax()

# # Step 2: Create a dictionary to store the latest correction rows
# latest_corrections = general_df_sorted.loc[latest_correction_idx]

# # Define the age groups you're interested in
# age_groups = ['15-19_yrs', '20-24_yrs', '25-29_yrs', '30-34_yrs', '35-39_yrs', '40-44_yrs', '45-49_yrs', '50+_yrs']

# # Columns to update for each age group
# columns_to_update = ['mc', 'sgDisposable', 'sgReusable', 'shangring', 'fu', 'hivPositive', 'hivNegative']

# # DataFrame to store the final results
# final_df = general_df_sorted.copy()

# # Step 1: Iterate through submission rows
# for idx, row in general_df_sorted[general_df_sorted['formType'] == 'submission'].iterrows():
#     facility, month, year = row['facilityName'], row['recordingMonth'], row['year']

#     # Step 2: Find corresponding correction row
#     correction_row = latest_corrections[
#         (latest_corrections['facilityName'] == facility) &
#         (latest_corrections['recordingMonth'] == month) &
#         (latest_corrections['year'] == year)
#     ]

#     print(correction_row)
#     if not correction_row.empty:  # If a correction row exists
#         # Step 3: Check if a row exists in lookupTable

#         lookup_match = lookupTable[
#             (lookupTable['facilityName'] == facility) &
#             (lookupTable['recordingMonth'] == month) &
#             (lookupTable['year'] == year)
#         ]

#         print(lookup_match)

#         if not lookup_match.empty:
#             # Apply corrections to lookupTable row instead of submission row
#             target_idx = lookup_match.index[0]  # Get index of matched row
#             target_df = lookupTable  # Apply updates to lookupTable
            
#             # Step 4: Update the matched row in lookupTable
#             age_groups_change = correction_row['ageGroupsChange'].values[0]
#             for age_group in age_groups:
#                 if age_group in age_groups_change:
#                     for column_prefix in columns_to_update:
#                         column_name = f'{column_prefix}{age_group.split("_")[0]}'
#                         target_df.at[target_idx, column_name] = correction_row[column_name].values[0]

#             # NEW LOGIC FOR 'total_mcs_referred_for_srh_services' UPDATE (LOOKUPTABLE PATH)
#             if 'copy-1-of-LinkagesSelection' in correction_row.columns and 'referral_srh' in correction_row['copy-1-of-LinkagesSelection'].values[0]:
#                 if 'total_mcs_referred_for_srh_services' in correction_row.columns:
#                     target_df.at[target_idx, 'total_mcs_referred_for_srh_services'] = correction_row['total_mcs_referred_for_srh_services'].values[0]

#             if 'copy-1-of-LinkagesSelection' in correction_row.columns and 'referral_sti' in correction_row['copy-1-of-LinkagesSelection'].values[0]:
#                 if 'total_mcs_referred_for_sti_services' in correction_row.columns:
#                     target_df.at[target_idx, 'total_mcs_referred_for_sti_services'] = correction_row['total_mcs_referred_for_sti_services'].values[0]


#             # Remove the submission row from final dataframe
#             final_df.drop(index=idx, inplace=True)

#             # Add the updated row from lookupTable to final dataframe
#             final_df = pd.concat([final_df, lookupTable.loc[[target_idx]]], ignore_index=True)

#         else:
#             # No match in lookupTable, apply changes to the original submission row
#             print("working here")

#             age_groups_change = correction_row['ageGroupsChange'].values[0]
#             for age_group in age_groups:
#                 if age_group in age_groups_change:
#                     for column_prefix in columns_to_update:
#                         column_name = f'{column_prefix}{age_group.split("_")[0]}'
#                         final_df.at[idx, column_name] = correction_row[column_name].values[0]

#             # NEW LOGIC FOR 'total_mcs_referred_for_srh_services' UPDATE (ORIGINAL SUBMISSION PATH)
#             if 'copy-1-of-LinkagesSelection' in correction_row.columns and 'referral_srh' in correction_row['copy-1-of-LinkagesSelection'].values[0]:
#                 if 'total_mcs_referred_for_srh_services' in correction_row.columns:
#                     final_df.at[idx, 'total_mcs_referred_for_srh_services'] = correction_row['total_mcs_referred_for_srh_services'].values[0] 


#             if 'copy-1-of-LinkagesSelection' in correction_row.columns and 'referral_sti' in correction_row['copy-1-of-LinkagesSelection'].values[0]:
#                 if 'total_mcs_referred_for_sti_services' in correction_row.columns:
#                     final_df.at[target_idx, 'total_mcs_referred_for_sti_services'] = correction_row['total_mcs_referred_for_sti_services'].values[0]






# # Step 5: Remove correction rows from final dataframe
# final_df = final_df[~final_df.index.isin(latest_corrections.index)]
# final_df = final_df[final_df['formType']!='correction']

# # Reset index to maintain consistency
# final_df = general_df_sorted
# final_df.reset_index(drop=True, inplace=True)


# correction_df = final_df[final_df['formType'] == 'correction']

# # Find the most recent 'correction' row per group
# # You can change 'ageGroupsChange' to another column if it's not a datetime or relevant
# latest_corrections = correction_df.drop_duplicates(subset=['facilityName', 'recordingMonth', 'year'])

# # Resulting DataFrame of the most recent correction per group
# latest_corrections.reset_index(drop=True, inplace=True)

# latest_corrections.to_excel("corrections.xlsx")

# final_df = final_df[~final_df["facilityName"].isin(["Masikati", "Murambi"])]

# # zazicDF = final_df[final_df['formType'] != 'correction'].copy()
# # zazicDF_updated = zazicDF.copy()

# # for _, row in latest_corrections.iterrows():


# # Save final result
# # zazicDF_updated.to_excel("data.xlsx", index=False)
# # Now save the DataFrame to Excel
# with pd.ExcelWriter('data.xlsx', engine='xlsxwriter') as writer:
#     # Save the sorted general data to the first sheet
#     final_df.to_excel(writer, sheet_name='Statistics', index=False)
    
#     # Save the sorted AE data to a new sheet
#     ae_df.drop_duplicates().to_excel(writer, sheet_name='AEs', index=False)
    
#     # Save the flattened referrals data to a new sheet
#     referrals_df.drop_duplicates().to_excel(writer, sheet_name='Referrals', index=False)

In [106]:
# final_df